In [1]:
#### 4.2 Тестирование на наборе данных *Breast cancer*:
import mo.hw1.task4 as t4

In [2]:
def det_train_test_split(X, y, test_part):
    objects_num = X.shape[0]
    test_part = 0.3
    train_part = 1 - test_part
    train_size = int(test_part * objects_num)
    X_train, X_test, y_train, y_test = [], [], [], []
    for i in range(train_size):
        X_train.append(X[i])
        y_train.append(y[i])
    for i in range(train_size, objects_num):
        X_test.append(X[i])
        y_test.append(y[i])
    X_train = t4.np.array(X_train)
    y_train = t4.np.array(y_train)
    X_test = t4.np.array(X_test)
    y_test = t4.np.array(y_test)
    return X_train, X_test, y_train, y_test


def calc_score(X_train, X_test, y_train, y_test, lmd, w_begin, eps, max_iter_num, logistic_method, dec_lim_method):
    w = logistic_method(X_train, y_train, lmd, w_begin, eps, max_iter_num)
    dec_lim = dec_lim_method(X_train, w)
    classifier = t4.get_classifier(w, dec_lim)

    test_objects_num = X_test.shape[0]
    conf_matrix = [[0] * 2 for _ in range(2)]
    for i in range(test_objects_num):
        y_real = y_test[i]
        y_pred = classifier(X_test[i])
        conf_matrix[y_pred][y_real] += 1

    tp, fp, fn, tn = conf_matrix[0][0], conf_matrix[0][1], conf_matrix[1][0], conf_matrix[1][1]
    accuracy = (tp + tn) / (tp + fp + fn + tn)
    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn)
    f_score = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    return conf_matrix, accuracy, precision, recall, f_score


def print_score(title, conf_matrix, accuracy, precision, recall, f_score):
    if title is not None:
        print(title)
    print('Confusion matrix')
    print(t4.pd.DataFrame(conf_matrix))
    print('Accuracy: {}'.format(accuracy))
    print('Precision: {}'.format(precision))
    print('Recall: {}'.format(recall))
    print('F-score: {}'.format(f_score))


def dec_lim_mean(X_train, w):
    train_objects_num, _ = X_train.shape
    sum_ = 0
    for i in range(train_objects_num):
        for j in range(len(w)):
            sum_ += w[j] * X_train[i][j]
    dec_lim = sum_ / train_objects_num
    return dec_lim


lgs_X, lgs_y = t4.datasets.load_breast_cancer(return_X_y=True)
lgs_X_train, lgs_X_test, lgs_y_train, lgs_y_test = det_train_test_split(lgs_X, lgs_y, 0.3)
lgs_lmd = 1e-2
lgs_w_begin = [
    -0.00017546, -0.00701235, 0.00737026, -0.005251, -0.00884388, -0.01521721,
    0.00714003, -0.01315466, -0.00755301, 0.00589339, 0.00822666, -0.00387729,
    0.00922983, 0.00066522, 0.01494301, 0.01284885, -0.0039456, 0.00708928,
    0.00065247, -0.01562194, -0.00127419, -0.01482304, 0.01539881, 0.01018855,
    -0.00404309, 0.00450084, -0.00044222, -0.00451975, -0.00546663, -0.00670873
]
lgs_eps = 1e-2
lgs_max_iter_num = 10

calc_score_applied = t4.partial(calc_score, lgs_X_train, lgs_X_test, lgs_y_train, lgs_y_test, lgs_lmd, lgs_w_begin, lgs_eps, lgs_max_iter_num)
grd_conf_matrix, grd_accuracy, grd_precision, grd_recall, grd_f_score = calc_score_applied(t4.logistic_gradient, dec_lim_mean)
print_score(None, grd_conf_matrix, grd_accuracy, grd_precision, grd_recall, grd_f_score)

Confusion matrix
    0    1
0  99    2
1  24  274
Accuracy: 0.9348370927318296
Precision: 0.9801980198019802
Recall: 0.8048780487804879
F-score: 0.8839285714285714
